In [1]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
import tensorflow as tf
from sklearn import preprocessing
import numpy as np
import random
import matplotlib.pyplot as plt
import sys
from utils.CLR import*
from utils.LoadData import load_CW_Source
from Model.CNN import cnn_classifier
from sklearn.utils import shuffle

d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
d:\VirtualEnvironment\Anaconda3\envs\TF-krease\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


ModuleNotFoundError: No module named 'utils'

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
def train_model(X_profiling, Y_profiling, X_test, Y_test, model, save_file_name, epochs=150, batch_size=100,
                max_lr=1e-3):

    # Save model every epoch
    save_model = ModelCheckpoint(
                    filepath=save_file_name,
                    monitor='val_loss',
                    mode='min',
                    save_best_only=True)

    # Get the input layer shape
    input_layer_shape = model.get_layer(index=0).input_shape[0]

    # Sanity check
    if input_layer_shape[1] != len(X_profiling[0]):
        print("Error: model input shape %d instead of %d is not expected ..." % (
        input_layer_shape[1], len(X_profiling[0])))
        sys.exit(-1)
    Reshaped_X_profiling, Reshaped_X_test = X_profiling.reshape(
        (X_profiling.shape[0], X_profiling.shape[1], 1)), X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # One Cycle Policy
    lr_manager = OneCycleLR(max_lr=max_lr, end_percentage=0.2, scale_percentage=0.1, maximum_momentum=None,
                            minimum_momentum=None, verbose=True)

    callbacks = [save_model, lr_manager]

    history = model.fit(x=Reshaped_X_profiling, y=to_categorical(Y_profiling, num_classes=256),
                        validation_data=(Reshaped_X_test, to_categorical(Y_test, num_classes=256)),
                        batch_size=batch_size, verbose=1, epochs=epochs, callbacks=callbacks)
    return history

<b> Trained on the dataset AES_device2

In [1]:
#region Configuration
TRAINING_CONFIG = {
    'bytes_to_train': range(0, 16),        # Process all 16 AES state bytes
    'input_size': 800,                    # Number of features per trace
    'learning_rate': 5e-3,                # Initial learning rate
    'batch_size': 512,                      # Training batch size
    'epochs': 100,                         # Training epochs        
    'save_file_name': '../Model/', # Model save location
}
#endregion

#region Main Training Loop
for byte_idx in TRAINING_CONFIG['bytes_to_train']:
    #region Session Management
    tf.keras.backend.clear_session()  # Prevent model parameter leakage
    #endregion

    #region Data Loading
    try:
        # Load byte-specific dataset
        (x_train, y_train, 
         x_test, y_test, 
         c_train, c_test) = load_CW_Source(in_file='../Dataset/AES_device2/',sec=18000,byte=byte_idx)
        
    except Exception as e:
        print(f"Data loading failed for byte {byte_idx}: {str(e)}")
        continue
    #endregion


    # Shuffle training data
    x_train, y_train = shuffle(x_train, y_train)
  


    #region Data Preprocessing
    # 1. Standardization
    std_scaler = preprocessing.StandardScaler()
    x_train = std_scaler.fit_transform(x_train.astype('float32'))
    x_test = std_scaler.transform(x_test.astype('float32'))

    # 2. Normalization
    minmax_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
    x_train = minmax_scaler.fit_transform(x_train)
    x_test= minmax_scaler.transform(x_test)

    #endregion.


    try:
        # Model initialization
        model = cnn_classifier(
            input_size=TRAINING_CONFIG['input_size'],
            learning_rate=TRAINING_CONFIG['learning_rate']
        )
        model.summary()
        
        # Training execution
        print(f"\n############### Training Byte {byte_idx} #################\n")
        history = train_model(
            x_train, y_train,
            x_test, y_test,
            model=model,
            save_file_name=f"{TRAINING_CONFIG['save_file_name']}Source_Model_byte{byte_idx}_D1.h5",
            epochs=TRAINING_CONFIG['epochs'],
            batch_size=TRAINING_CONFIG['batch_size'],
            max_lr=TRAINING_CONFIG['learning_rate']
        )
        
        print(f"\n############### Byte {byte_idx} Training Completed #################\n")
        
    except Exception as e:
        print(f"Training failed for byte {byte_idx}: {str(e)}")
        continue